In [1]:
import numpy as np
import cv2

1. Function to calculate IoU

In [3]:
def calculate_iou(bbox1, bbox2): 
    # box shape (x1,y1,x2,y2) (x1,y1) is top left corner, (x2,y2) is bottom right corner
    
    # Calculate the intersection area
    intersection_width = min(bbox1[2], bbox2[2]) - max(bbox1[0], bbox2[0])
    intersection_height = min(bbox1[3], bbox2[3]) - max(bbox1[1], bbox2[1])
    intersection_area = max(0, intersection_width) * max(0, intersection_height)

    # Calculate the union area
    bbox1_area = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    bbox2_area = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    union_area = bbox1_area + bbox2_area - intersection_area

    # Calculate the IoU
    iou = intersection_area / union_area

    return iou

2. Function to eliminate the parent class of object

In [4]:
def eliminate_parent_class(classes, class_hierarchy):
    children = []
    parents = []
    for child, parent in class_hierarchy.items():
        children.append(child)
        parents.append(parent)

    children = [child for child in children if child not in parents]
    ind_classes = [i for i in range(len(classes)) if classes[i] in children]

    return ind_classes

3. Non maximum suppression algorithm

In [5]:
def nms(bboxes, scores, classes, class_hierarchy, score_threshold, iou_threshold):

    #eliminate the parent class of object
    index = eliminate_parent_class(classes, class_hierarchy)

    bboxes = bboxes[index]
    scores = scores[index]
    classes = classes[index]

    # Sort the boxes based on scores in descending order
    scores = scores[scores > score_threshold]
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    sorted_scores = [scores[i] for i in sorted_indices]
    sorted_boxes = [bboxes[i] for i in sorted_indices]
    sorted_classes = [classes[i] for i in sorted_indices]
    
    # Initialize an empty list to store the selected boxes
    selected_boxes = []
    selected_scores = []
    selected_classes = []
    
    while sorted_boxes:
        # Select the box with the highest score as the reference box
        reference_box = sorted_boxes[0]
        reference_class = sorted_classes[0]
        reference_score = sorted_scores[0]

        selected_boxes.append(reference_box)
        selected_scores.append(reference_score)
        selected_classes.append(reference_class)
        
        # Calculate the IoU between the reference box and other boxes
        ious = [calculate_iou(reference_box, box) for box in sorted_boxes[1:]]
    
        # Remove boxes with IoU greater than the threshold
        sorted_boxes = [box for i, box in enumerate(sorted_boxes[1:]) if ious[i] < iou_threshold]
        sorted_classes = [cls for i, cls in enumerate(sorted_classes[1:]) if ious[i] < iou_threshold]
        sorted_scores = [sc for i, sc in enumerate(sorted_scores[1:]) if ious[i] < iou_threshold]


    return selected_boxes, selected_scores, selected_classes

4. Examle

In [6]:
bboxes = np.array([[100, 200, 500, 600],[110, 210, 510, 610],[105, 230, 520, 590],
                   [120, 180, 550, 650], [290, 100, 600, 650], 
                   [300, 120, 610, 560],[310, 110, 600, 580],[305, 130, 620, 570]])
scores = np.array([0.9, 0.89, 0.8, 0.85, 0.87, 0.86, 0.8, 0.88])
classes = np.array(['Apple','Apple','Apple', 'Fruit', 'Fruit', "Apple",'Apple','Apple'])
class_hierarchy = {
    'Apple': 'Fruit'
}
score_threshold = 0.6
iou_threshold = 0.6

In [8]:
result = nms(bboxes, scores, classes, class_hierarchy, score_threshold, iou_threshold)
print("selected_boxes:",result[0])
print("selected_scores:",result[1])
print("selected_classes:",result[2])

selected_boxes: [array([100, 200, 500, 600]), array([305, 130, 620, 570])]
selected_scores: [0.9, 0.88]
selected_classes: ['Apple', 'Apple']


In [9]:
image = np.zeros((1000, 1000, 3), dtype=np.uint8)

# Define the rectangles parameters
rectangles = [
    {'x1': 100, 'y1': 200, 'x2': 500, 'y2': 600, 'color': (0, 0, 200), 'label': 'Apple'},
    {'x1': 110, 'y1': 210, 'x2': 510, 'y2': 610, 'color': (0, 0, 200), 'label': 'Apple'},
    {'x1': 105, 'y1': 230, 'x2': 520, 'y2': 590, 'color': (0, 0, 200), 'label': 'Apple'},
    {'x1': 120, 'y1': 180, 'x2': 510, 'y2': 610, 'color': (255, 255, 255), 'label': 'Fruit'},  
    {'x1': 290, 'y1': 100, 'x2': 600, 'y2': 550, 'color': (255, 255, 255), 'label': 'Fruit'},   
    {'x1': 300, 'y1': 120, 'x2': 610, 'y2': 560, 'color': (0, 0, 200), 'label': 'Apple'},
    {'x1': 310, 'y1': 110, 'x2': 600, 'y2': 580, 'color': (0, 0, 200), 'label': 'Apple'},
    {'x1': 305, 'y1': 130, 'x2': 620, 'y2': 570, 'color': (0, 0, 200), 'label': 'Apple'} 

]

# Draw the rectangles on the image
for rectangle in rectangles:
    x1, y1 = rectangle['x1'], rectangle['y1']
    x2, y2 = rectangle['x2'], rectangle['y2']
    color = rectangle['color']
    thickness = 2
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    
    # Add annotation
    label = rectangle['label']
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Display the image
cv2.imshow("Rectangles", image)
cv2.waitKey(0)

-1

In [10]:
image = np.zeros((1000, 1000, 3), dtype=np.uint8)

# Define the rectangles parameters
rectangles = [
    {'x1': 100, 'y1': 200, 'x2': 500, 'y2': 600, 'color': (0, 0, 200), 'label': 'Apple'},
    # {'x1': 110, 'y1': 210, 'x2': 510, 'y2': 610, 'color': (0, 0, 200), 'label': 'Apple'},
    # {'x1': 105, 'y1': 230, 'x2': 520, 'y2': 590, 'color': (0, 0, 200), 'label': 'Apple'},
    # {'x1': 120, 'y1': 180, 'x2': 510, 'y2': 610, 'color': (255, 255, 255), 'label': 'Fruit'},  
    # {'x1': 290, 'y1': 100, 'x2': 600, 'y2': 550, 'color': (255, 255, 255), 'label': 'Fruit'},   
    # {'x1': 300, 'y1': 120, 'x2': 610, 'y2': 560, 'color': (0, 0, 200), 'label': 'Apple'},
    # {'x1': 310, 'y1': 110, 'x2': 600, 'y2': 580, 'color': (0, 0, 200), 'label': 'Apple'},
    {'x1': 305, 'y1': 130, 'x2': 620, 'y2': 570, 'color': (0, 0, 200), 'label': 'Apple'} 

]

# Draw the rectangles on the image
for rectangle in rectangles:
    x1, y1 = rectangle['x1'], rectangle['y1']
    x2, y2 = rectangle['x2'], rectangle['y2']
    color = rectangle['color']
    thickness = 2
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    
    # Add annotation
    label = rectangle['label']
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Display the image
cv2.imshow("Rectangles", image)
cv2.waitKey(0)

-1